In [104]:
import pandas as pd
import numpy as np
import os
import scipy.linalg as linalg
from pulp import *
import csv
import itertools
from tqdm import tqdm
import statsmodels.stats.moment_helpers as stats
import scipy
import matplotlib.pyplot as plt
import math
import matplotlib.mlab as mlab
from datetime import datetime
os.chdir('/home/valesco/Datasets/NFL_projs')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [118]:
def home_away(x):
    if '@' in x:
        return 'away'
    else:
        return 'home'

proj_df = pd.read_csv('week4_dk_proj.csv')
cov_df = pd.read_csv('nfl_covariance_matrix.csv', index_col = 0)

#number of games in spreadsheet plus one
num_of_games = 17


proj_df['matchup'] = tuple(zip(proj_df['team'], proj_df['opponent']))
#proj_df['salary'] = proj_df['salary'].apply(lambda x: x.replace('$',''))
#proj_df['salary'] = proj_df['salary'].apply(lambda x: float(x.replace('K', '')))
#proj_df['salary'] = proj_df['salary'].apply(lambda x: x * 1000)
proj_df['rank'] = proj_df.groupby(['team', 'position'])['points_avg'].rank(ascending = False, method = 'max')

proj_df['rank'].fillna(value = 9999, inplace = True)
proj_df['salary'].fillna(value = 9999, inplace = True)
proj_df['points_avg'].fillna(value = -9.999, inplace = True)
proj_df['rank'] = proj_df['rank'].apply(lambda x: str(int(x)))
proj_df['home_away'] = proj_df['opponent'].apply(lambda x: home_away(x))
proj_df['rank'] = proj_df['home_away'] + '_' + proj_df['position'] + proj_df['rank']

proj_df.to_csv('2016_week4_proj_process.csv')
proj_df.head()

,player_id,name,salary,team,opponent,points_avg,points_floor,floor_pct,floor_pt_per_dpct,points_ceiling,ceiling_pct,consistency,position,position_count,position_id,positions,adj_ypa,event_team_id,expected_sack_pct,Properties.ImpPts,Properties.InterceptionPct,Properties.IsProjected,Properties.MktShrPct,Properties.Month_FPOpp,Properties.Month_FPSnap,...,Properties.SackPct,Properties.Salary,Properties.Salary_Movement,Properties.Score,Properties.Season_FPOpp,Properties.Season_FPSnap,Properties.Season_PPG,Properties.Season_PPG_Percentile,Properties.Season_Plus_Minus,Properties.Season_Salary_Change,Properties.Season_X0,Properties.Season_X1,Properties.Season_X2,Properties.Site_Salary,Properties.SourceId,Properties.TakeawayPct,Properties.Team,Properties.TouchdownAllowedPct,Properties.Upside,Properties.Vegas,Properties.YardsPerPassingAttempt,Score,matchup,rank,home_away
0,969,Kamar Aiken,3000,BAL,OAK,2.7,1.4,25,25,4.0,22,76,WR,1,103,['WR'],NaN,32863802,57,5.98,1.62,True,80.0,0.0,0.05,...,7.09,3000,99,47.0,0.0,0.22,11.6,73,2.2,-1700,20,53,26,99,4,0.93,BAL,4.13,47,77,7.08,47.0,"(BAL, OAK)",home_WR4,home
1,1007,Dwayne Allen,3800,IND,@JAX,10.0,6.7,82,82,13.3,81,51,TE,1,104,['TE'],NaN,32863101,31,6.85,2.50,True,38.0,0.0,0.17,...,5.84,3800,7,37.0,0.0,0.08,3.6,44,-1.4,600,57,35,7,79,4,1.46,IND,4.36,18,84,7.50,37.0,"(IND, @JAX)",away_TE1,away
2,1047,Danny Amendola,3600,NE,BUF,4.6,2.6,35,35,6.6,32,58,WR,1,103,['WR'],36.0,32863302,35,7.35,1.62,True,71.0,0.0,0.63,...,6.63,3600,90,75.0,1.6,0.33,11.7,74,3.0,-600,14,42,42,80,4,1.01,NE,2.89,75,50,6.85,75.0,"(NE, BUF)",home_WR3,home
3,1055,C.J. Anderson,6500,DEN,@TB,18.0,12.8,95,95,23.2,94,41,RB,1,102,['RB'],NaN,32863901,14,14.71,1.91,True,87.0,0.0,0.38,...,9.22,6500,11,84.0,0.0,0.38,12.4,83,2.9,600,33,26,40,22,4,1.12,DEN,3.52,68,70,6.18,84.0,"(DEN, @TB)",away_RB1,away
4,1124,Matt Asiata,3300,MIN,NYG,10.1,6.7,69,69,13.5,69,85,RB,1,102,['RB'],NaN,32864602,11,5.63,2.88,True,25.0,0.0,0.20,...,9.37,3300,74,35.0,0.0,0.20,2.6,29,-2.3,0,37,62,0,96,4,1.68,MIN,3.45,21,72,7.26,35.0,"(MIN, NYG)",home_RB2,home


In [119]:
away_teams = proj_df['team'].loc[proj_df['home_away'] == 'away'].unique()

player_cols = []
count = 0

for team in away_teams:
    temp1 = proj_df.loc[proj_df['team'] == team]
    temp2 = proj_df.loc[proj_df['opponent'] == team]
    temp_df = pd.concat([temp1, temp2], axis = 0)
    
    cols = cov_df.columns

    player_ls = []
    for c in cols:
        player = pd.DataFrame(temp_df.loc[temp_df['rank'] == c])
        
        if len(player) < 1:
            name = 'Not Available_' + str(count)
            count += 1
        else:
            name = player.iloc[0]['name']
        player_ls.append(name)
        
    player_cols.extend(player_ls)
    

arr_cov = np.asarray(cov_df)
for i in range(1, 15):
    arr_new = np.asarray(cov_df)
    arr_cov = linalg.block_diag(arr_cov, arr_new)
    
comp_cov_df = pd.DataFrame(data = arr_cov, index = player_cols, columns = player_cols)
comp_cov_df.head()

,Jacksonville Defense,Blake Bortles,T.J. Yeldon,Chris Ivory,Julius Thomas,Allen Robinson,Allen Hurns,Marqise Lee,Indianapolis Defense,Andrew Luck,Frank Gore,Josh Ferguson,Dwayne Allen,T.Y. Hilton,Phillip Dorsett,Chester Rogers,Tampa Bay Defense,Jameis Winston,Charles Sims,Jacquizz Rodgers,Cameron Brate,Mike Evans,Adam Humphries,Vincent Jackson,Denver Defense,...,Jaelen Strong,Tennessee Defense,Marcus Mariota,DeMarco Murray,Derrick Henry,Delanie Walker,Tajae Sharpe,Rishard Matthews,Kendall Wright,Cincinnati Defense,Andy Dalton,Giovani Bernard,Jeremy Hill,C.J. Uzomah,A.J. Green,Not Available_3,Brandon LaFell,Miami Defense,Ryan Tannehill,Kenyan Drake,Jay Ajayi,Dion Sims,Jarvis Landry,DeVante Parker,Kenny Stills
Jacksonville Defense,1.00,-0.08,0.11,0.03,-0.08,0.01,0.00,-0.09,-0.24,-0.41,-0.25,-0.13,-0.09,-0.18,-0.12,-0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Blake Bortles,-0.08,1.00,0.08,0.06,0.33,0.46,0.33,0.28,-0.41,0.20,0.03,-0.01,0.09,0.09,0.07,0.13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T.J. Yeldon,0.11,0.08,1.00,-0.13,-0.06,0.03,-0.03,-0.07,-0.25,0.03,-0.01,-0.06,0.03,0.00,0.01,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chris Ivory,0.03,0.06,-0.13,1.00,0.04,-0.03,-0.07,0.00,-0.13,-0.01,-0.06,-0.05,0.00,-0.02,-0.03,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Julius Thomas,-0.08,0.33,-0.06,0.04,1.00,-0.04,-0.06,0.00,-0.09,0.09,0.03,0.00,0.04,0.02,0.04,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
position_std = {'QB': 8.096014, 'WR': 9.813933, 'RB': 8.702397, 'TE': 8.096014, 'D': 6.09265}
position_mu = {'QB': 19.861207, 'WR': 12.67457, 'RB': 12.900904, 'TE': 12.50152, 'D': 7.447265}

In [121]:
matchups = proj_df['matchup'].unique()
games = []
for game in matchups:
    if '@' in game[0] or '@' in game[1]:
        temp1 = game[0].replace('@','')
        temp2 = game[1].replace('@','')
        games.append((temp1, temp2))
    
#games = games[1:16]

combo_df = pd.DataFrame(columns = ['p1_name', 'p1_points', 'p1_pos', 'p1_salary', 'p1_rank', 'p1_std', 
                                   'p2_name', 'p2_points', 'p2_pos', 'p2_salary', 'p2_rank', 'p2_std',
                                   'p3_name', 'p3_points', 'p3_pos', 'p3_salary', 'p3_rank', 'p3_std',
                                   'p4_name', 'p4_points', 'p4_pos', 'p4_salary', 'p4_rank', 'p4_std',
                                   '1_2corr', '1_3corr', '1_4corr', '2_3corr', '2_4corr', '3_4corr',
                                   'combo_salary', 'combo_points', 'qb_count', 'wr_count',
                                   'rb_count', 'te_count', 'dst_count'
                                   ])
row_count = 0
for game in tqdm(games):
    
    temp_home = proj_df.loc[proj_df['team'] == game[0]]    
    temp_away = proj_df.loc[proj_df['team'] == game[1]]
    temp = pd.concat([temp_home, temp_away], axis = 0)
    temp = temp.loc[temp['points_avg'] > 7]

    player_ls = temp['name'].values

    combos = []

    for i in range(2,5):
        for tup in itertools.combinations(player_ls, i):
            combos.append(tup)

    for combo in combos:
        for c in range(len(combo)):
            if c == 0:
                player_temp = temp.loc[temp['name'] == combo[c]]
                name1 = player_temp.iloc[0]['name']
                points1 = player_temp.iloc[0]['points_avg']
                pos1 = player_temp.iloc[0]['position']
                salary1 = player_temp.iloc[0]['salary']
                rank1 = player_temp.iloc[0]['rank']
                std1 = position_std[pos1]
            if c == 1:
                player_temp = temp.loc[temp['name'] == combo[c]]
                name2 = player_temp.iloc[0]['name']
                points2 = player_temp.iloc[0]['points_avg']
                pos2 = player_temp.iloc[0]['position']
                salary2 = player_temp.iloc[0]['salary']
                rank2 = player_temp.iloc[0]['rank']
                std2 = position_std[pos2]
            if len(combo) > 2:
                if c == 2:
                    player_temp = temp.loc[temp['name'] == combo[c]]
                    name3 = player_temp.iloc[0]['name']
                    points3 = player_temp.iloc[0]['points_avg']
                    pos3 = player_temp.iloc[0]['position']
                    salary3 = player_temp.iloc[0]['salary']
                    rank3 = player_temp.iloc[0]['rank']
                    std3 = position_std[pos3]
            else:
                name3 = 'null'
                points3 = 0
                pos3 = 'null'
                salary3 = 0
                rank3 = 'null'
                std3 = 0
            if len(combo) > 3:
                if c == 3:
                    player_temp = temp.loc[temp['name'] == combo[c]]
                    name4 = player_temp.iloc[0]['name']
                    points4 = player_temp.iloc[0]['points_avg']
                    pos4 = player_temp.iloc[0]['position']
                    salary4 = player_temp.iloc[0]['salary']
                    rank4 = player_temp.iloc[0]['rank']
                    std4 = position_std[pos4]
            else:
                name4 = 'null'
                points4 = 0
                pos4 = 'null'
                salary4 = 0
                rank4 = 'null'
                std4 = 0

                
        try:       
            corr1_2 = comp_cov_df.loc[name1][name2]
        except:
            corr1_2 = 0
        if name3 != 'null':
            try:
                corr1_3 = comp_cov_df.loc[name1][name3]
                corr2_3 = comp_cov_df.loc[name2][name3]
            except:
                corr1_3 = 0
                corr2_3 = 0
        else:
            corr1_3 = 0
            corr2_3 = 0
        if name4 != 'null':
            try:
                corr1_4 = comp_cov_df.loc[name1][name4]
                corr2_4 = comp_cov_df.loc[name2][name4]
                corr3_4 = comp_cov_df.loc[name3][name4]
            except:
                corr1_4 = 0
                corr2_4 = 0
                corr3_4 = 0
        else:
            corr1_4 = 0
            corr2_4 = 0
            corr3_4 = 0
        
        combo_salary = salary1 + salary2 + salary3 + salary4
        combo_points = points1 + points2 + points3 + points4
        
        pos_ls = [pos1, pos2, pos3, pos4]
        
        qb_count = 0
        wr_count = 0
        rb_count = 0
        te_count = 0
        dst_count = 0
        
        for pos in pos_ls:
            if pos == 'QB':
                qb_count += 1
            elif pos == 'WR':
                wr_count += 1
            elif pos == 'RB':
                rb_count += 1
            elif pos == 'TE':
                te_count += 1
            elif pos == 'D':
                dst_count += 1
            elif pos == 'null':
                nope = 'nope'
            else:
                print('HOOLLLLAAA!!!')
                
        if qb_count > 1:
            pass
        elif dst_count > 1:
            pass
        elif rb_count > 3:
            pass
        elif te_count > 2:
            pass
        else:
            combo_df.loc[row_count] = [name1, points1, pos1, salary1, rank1, std1,
                               name2, points2, pos2, salary2, rank2, std2,
                               name3, points3, pos3, salary3, rank3, std3,
                               name4, points4, pos4, salary4, rank4, std4,
                               corr1_2, corr1_3, corr1_4, corr2_3, corr2_4, corr3_4,
                               combo_salary, combo_points, qb_count, wr_count, rb_count,
                               te_count, dst_count]
        row_count += 1

100%|██████████| 15/15 [05:51<00:00, 28.28s/it]


In [122]:
for index, row in tqdm(combo_df.iterrows()):
    player_count = row['qb_count'] + row['wr_count'] + row['rb_count'] + row['te_count'] + row['dst_count']
    
    if player_count == 2:
        corr1_2 = row['1_2corr']
        
        temp_mu = row['p1_points'] + row['p2_points']
        temp_array = np.array([[1, corr1_2], [corr1_2, 1]])
        temp_std = [row['p1_std'], row['p2_std']]
        cov_matrix = stats.corr2cov(temp_array, temp_std)
        temp_var = row['p1_std'] ** 2 + row['p2_std'] ** 2 + 2 * (cov_matrix[1,0])
        temp_dist = scipy.stats.norm(temp_mu, np.sqrt(temp_var))
        
        temp_ppf = .90
    elif player_count == 3:
        corr1_2 = row['1_2corr']
        corr1_3 = row['1_3corr']
        corr2_3 = row['2_3corr']
        
        temp_mu = row['p1_points'] + row['p2_points'] + row['p3_points']
        temp_array = np.array([[1, corr1_2, corr1_3],[corr1_2, 1, corr2_3], [corr1_3, corr2_3, 1]])
        temp_std = [row['p1_std'], row['p2_std'], row['p3_std']]
        cov_matrix = stats.corr2cov(temp_array, temp_std)
        temp_var = row['p1_std'] ** 2 + row['p2_std'] ** 2 + row['p3_std'] ** 2 + 2 * (cov_matrix[1,0] + 
                                                            cov_matrix[2,0] + cov_matrix[2,1])
        temp_dist = scipy.stats.norm(temp_mu, np.sqrt(temp_var))
        
        temp_ppf = .95
    elif player_count == 4:
        corr1_2 = row['1_2corr']
        corr1_3 = row['1_3corr']
        corr1_4 = row['1_4corr']
        corr2_3 = row['2_3corr']
        corr2_4 = row['2_4corr']
        corr3_4 = row['3_4corr']
        p1_std = row['p1_std']
        p2_std = row['p2_std']
        p3_std = row['p3_std']
        p4_std = row['p4_std']
        
        temp_mu = row['p1_points'] + row['p2_points'] + row['p3_points'] + row['p4_points']
        temp_array = np.array([[1, corr1_2, corr1_3, corr1_4], [corr1_2, 1, corr2_3, corr2_4],
                              [corr1_3, corr2_3, 1, corr3_4], [corr1_4, corr2_4, corr3_4, 1]])
        temp_std = [p1_std, p2_std, p3_std, p4_std]
        cov_matrix = stats.corr2cov(temp_array, temp_std)
        temp_var = p1_std ** 2 + p2_std ** 2 + p3_std ** 2 + p4_std ** 2 + 2 * (cov_matrix[1,0] + cov_matrix[2, 0] + 
                                        cov_matrix[3,0] + cov_matrix[2, 1] + cov_matrix[3,1] + cov_matrix[3,2])
         
        temp_dist = scipy.stats.norm(temp_mu, np.sqrt(temp_var))
        
        temp_ppf = .975
    else:
        print('HOOOOLLLLLAAAA')
        
    combo_df.loc[index, 'combo_ppf'] = scipy.stats.norm(temp_mu, np.sqrt(temp_var)).ppf(temp_ppf)

14407it [00:39, 366.15it/s]


In [123]:
row_count = len(combo_df)

for index, row in proj_df.iterrows():
    p1_name = row['name']
    p1_points = row['points_avg']
    p1_pos = row['position']
    p1_salary = row['salary']
    p1_rank = row['rank']
    
    qb_count = 0
    wr_count = 0
    rb_count = 0
    te_count = 0 
    dst_count = 0
    
    if p1_pos == 'QB':
        qb_count = 1
        p1_std = position_std['QB']
    elif p1_pos == 'WR':
        wr_count = 1
        p1_std = position_std['WR']
    elif p1_pos == 'RB':
        rb_count = 1
        p1_std = position_std['RB']
    elif p1_pos == 'TE':
        te_count = 1
        p1_std = position_std['TE']
    elif p1_pos == 'D':
        dst_count = 1
        p1_std = position_std['D']
    else:
        print('HOOOLLAAAA')
        
    corr1_2 = 0
    corr1_3 = 0
    corr1_4 = 0
    corr2_3 = 0
    corr2_4 = 0
    corr3_4 = 0
    
    combo_salary = p1_salary
    combo_points = p1_points
    
    p1_ppf = scipy.stats.norm(p1_points, p1_std).ppf(.80)
    
    combo_df.loc[row_count] = [p1_name, p1_points, p1_pos, p1_salary, p1_rank, p1_std,
                               'null', 0, 'null', 0, 'null', 0,
                               'null', 0, 'null', 0, 'null', 0,
                               'null', 0, 'null', 0, 'null', 0,
                               corr1_2, corr1_3, corr1_4, corr2_3, corr2_4, corr3_4,
                               combo_salary, combo_points, qb_count, wr_count, rb_count,
                               te_count, dst_count, p1_ppf]
    
    row_count += 1

In [8]:
#combo_df['players'] = tuple(zip(combo_df['p1_name'], combo_df['p2_name'], combo_df['p3_name'], combo_df['p4_name']))

#dummies = combo_df['players'].str.join(sep = '*').str.get_dummies(sep = '*')

#combo_df = pd.concat([combo_df, dummies], axis = 1)

In [128]:
cols = combo_df.columns
cols = list(cols[39:])

#combo_df.drop('players', axis = 1, inplace = True)
combo_df.to_csv('week4_ppf.csv', index = False)

In [138]:
in_file = "week4_ppf.csv"
reader = csv.DictReader(open(in_file, 'r'))
player_dict = []
for i , row in enumerate(reader):
    team = {k: v for k, v in row.items()}
    team['id'] = i
    player_dict.append(team)
    
players = [player['id'] for player in player_dict]
teams = [0]

In [141]:
start = datetime.now()

players = [player['id'] for player in player_dict]            

opt = LpProblem('Optimizer', LpMaximize)
possible_assignments = LpVariable.dicts("possible", (teams, players), 0, 1, LpBinary)

#objective
opt += sum([float(player['combo_ppf']) * possible_assignments[team][player['id']] for player in player_dict for team in teams])

#set player max to 1 per team:
for col in cols:
    opt += lpSum([float(player[col]) * possible_assignments[team][player['id']] for player in player_dict]) <= 1, 'player' + col


#set constraints
for team in teams:
    opt += lpSum([float(player['combo_salary']) * possible_assignments[team][player['id']] for player in player_dict]) <= 50000
    opt += lpSum([float(player['qb_count']) * possible_assignments[team][player['id']] for player in player_dict]) == 1
    opt += lpSum([float(player['wr_count']) * possible_assignments[team][player['id']] for player in player_dict]) == 3
    opt += lpSum([float(player['rb_count']) * possible_assignments[team][player['id']] for player in player_dict]) == 3
    opt += lpSum([float(player['te_count']) * possible_assignments[team][player['id']] for player in player_dict]) == 1
    opt += lpSum([float(player['dst_count']) * possible_assignments[team][player['id']] for player in player_dict]) == 1
    opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['p1_name'] == 'New Orleans Defense']) == 0
    opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['p2_name'] == 'New Orleans Defense']) == 0
    opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['p3_name'] == 'New Orleans Defense']) == 0
    opt += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['p4_name'] == 'New Orleans Defense']) == 0
    #opt += lpSum([(float(player['dst_count']) + float(player['qb_count']) + float(player['wr_count'])
    #            + float(player['rb_count'])) * possible_assignments[team][player['id']] for player in player_dict]) == 8
    
    

opt.writeLP('fantasy_optimization.lp')
opt.solve()
status = LpStatus[opt.status]
selected = []

finish = datetime.now() - start
print(finish)

0:02:45.588659


In [142]:
print(status)

Optimal


In [143]:
team_dict = {}
for player in player_dict:
            if possible_assignments[0][player['id']].value() == 1.0:
                players = {}
                #print(player['player'] + ' salary: ' + player['salary'] + ' proj: ' + player['odds'])
                #opt_odds.append(float(player['odds']))
                p1_name = player['p1_name']
                p2_name = player['p2_name']
                p3_name = player['p3_name']
                p4_name = player['p4_name']
                
                p1_points = player['p1_points']
                p2_points = player['p2_points']
                p3_points = player['p3_points']
                p4_points = player['p4_points']
                
                p1_pos = player['p1_pos']
                p2_pos = player['p2_pos']
                p3_pos = player['p3_pos']
                p4_pos = player['p4_pos']
                
                p1_salary = player['p1_salary']
                p2_salary = player['p2_salary']
                p3_salary = player['p3_salary']
                p4_salary = player['p4_salary']
                
                p_ls = [p1_name, p2_name, p3_name, p4_name]
                
                for i, p in enumerate(p_ls):
                    if p != 'null':
                        if i == 0:
                            print('{}, {}, {}, {}'.format(p1_name, p1_points, p1_pos, p1_salary))
                        if i == 1:
                            print('{}, {}, {}, {}'.format(p2_name, p2_points, p2_pos, p2_salary))
                        if i == 2:
                            print('{}, {}, {}, {}'.format(p3_name, p3_points, p3_pos, p3_salary))
                        if i == 3:
                            print('{}, {}, {}, {}'.format(p4_name, p4_points, p4_pos, p4_salary))    
            
                    
           
                team_dict[player['id']] = players

Mark Ingram, 18.5, RB, 5900.0
Travis Benjamin, 18.2, WR, 5900.0
Melvin Gordon, 20.8, RB, 6300.0
Travis Benjamin, 18.2, WR, 5900.0
Philip Rivers, 23.3, QB, 6900.0
Melvin Gordon, 20.8, RB, 6300.0
Tyrell Williams, 14.1, WR, 4400.0
Travis Kelce, 15.5, TE, 4800.0
Carolina Defense, 9.8, D, 3400.0


In [145]:
combo_df['player_count'] = combo_df['qb_count'] + combo_df['wr_count'] + combo_df['rb_count'] + combo_df['te_count'] + combo_df['dst_count']

combo_df.sort_values(by = ['player_count', 'combo_ppf'], ascending = False, inplace = True)

In [147]:
combo_df.loc[(combo_df['dst_count'] == 1) & (combo_df['player_count'] == 3) & (combo_df['rb_count'] == 1)]

,p1_name,p1_points,p1_pos,p1_salary,p1_rank,p1_std,p2_name,p2_points,p2_pos,p2_salary,p2_rank,p2_std,p3_name,p3_points,p3_pos,p3_salary,p3_rank,p3_std,p4_name,p4_points,p4_pos,p4_salary,p4_rank,p4_std,1_2corr,1_3corr,1_4corr,2_3corr,2_4corr,3_4corr,combo_salary,combo_points,qb_count,wr_count,rb_count,te_count,dst_count,combo_ppf,player_count
11290,Le'Veon Bell,20.9,RB,7500.0,home_RB1,8.702397,Antonio Brown,25.3,WR,9700.0,home_WR1,9.813933,Pittsburgh Defense,7.5,D,2800.0,home_D1,6.092650,null,0.0,null,0.0,null,0.0,0.03,0.11,0.0,0.01,0.0,0.0,20000.0,53.7,0.0,1.0,1.0,0.0,1.0,78.490409,3.0
11246,Kansas City Defense,7.5,D,3000.0,away_D1,6.092650,Le'Veon Bell,20.9,RB,7500.0,home_RB1,8.702397,Antonio Brown,25.3,WR,9700.0,home_WR1,9.813933,null,0.0,null,0.0,null,0.0,-0.25,-0.18,0.0,0.03,0.0,0.0,20200.0,53.7,0.0,1.0,1.0,0.0,1.0,74.908658,3.0
2079,Carson Palmer,20.8,QB,6300.0,home_QB1,8.096014,Arizona Defense,8.4,D,3900.0,home_D1,6.092650,David Johnson,22.0,RB,7800.0,home_RB1,8.702397,null,0.0,null,0.0,null,0.0,-0.08,0.08,0.0,0.11,0.0,0.0,18000.0,51.2,1.0,0.0,1.0,0.0,1.0,74.077227,3.0
11294,Le'Veon Bell,20.9,RB,7500.0,home_RB1,8.702397,Ben Roethlisberger,22.6,QB,7100.0,home_QB1,8.096014,Pittsburgh Defense,7.5,D,2800.0,home_D1,6.092650,null,0.0,null,0.0,null,0.0,0.08,0.11,0.0,-0.08,0.0,0.0,17400.0,51.0,1.0,0.0,1.0,0.0,1.0,73.877227,3.0
2075,Larry Fitzgerald,18.2,WR,7200.0,home_WR1,9.813933,Arizona Defense,8.4,D,3900.0,home_D1,6.092650,David Johnson,22.0,RB,7800.0,home_RB1,8.702397,null,0.0,null,0.0,null,0.0,0.01,0.03,0.0,0.11,0.0,0.0,18900.0,48.6,0.0,1.0,1.0,0.0,1.0,73.390409,3.0
11169,Spencer Ware,14.5,RB,5700.0,away_RB1,8.702397,Kansas City Defense,7.5,D,3000.0,away_D1,6.092650,Antonio Brown,25.3,WR,9700.0,home_WR1,9.813933,null,0.0,null,0.0,null,0.0,0.11,0.00,0.0,-0.18,0.0,0.0,18400.0,47.3,0.0,1.0,1.0,0.0,1.0,70.521261,3.0
13923,DeAndre Hopkins,18.9,WR,8400.0,home_WR1,9.813933,Lamar Miller,19.1,RB,7000.0,home_RB1,8.702397,Houston Defense,7.6,D,3200.0,home_D1,6.092650,null,0.0,null,0.0,null,0.0,0.03,0.01,0.0,0.11,0.0,0.0,18600.0,45.6,0.0,1.0,1.0,0.0,1.0,70.390409,3.0
1164,C.J. Anderson,18.0,RB,6500.0,away_RB1,8.702397,Demaryius Thomas,17.7,WR,6700.0,away_WR1,9.813933,Denver Defense,9.3,D,3700.0,away_D1,6.092650,null,0.0,null,0.0,null,0.0,0.03,0.11,0.0,0.01,0.0,0.0,16900.0,45.0,0.0,1.0,1.0,0.0,1.0,69.790409,3.0
11085,Jeremy Maclin,18.0,WR,6000.0,away_WR1,9.813933,Le'Veon Bell,20.9,RB,7500.0,home_RB1,8.702397,Pittsburgh Defense,7.5,D,2800.0,home_D1,6.092650,null,0.0,null,0.0,null,0.0,0.00,-0.18,0.0,0.11,0.0,0.0,16300.0,46.4,0.0,1.0,1.0,0.0,1.0,69.621261,3.0
11190,Spencer Ware,14.5,RB,5700.0,away_RB1,8.702397,Antonio Brown,25.3,WR,9700.0,home_WR1,9.813933,Pittsburgh Defense,7.5,D,2800.0,home_D1,6.092650,null,0.0,null,0.0,null,0.0,0.00,-0.25,0.0,0.01,0.0,0.0,18200.0,47.3,0.0,1.0,1.0,0.0,1.0,69.602839,3.0
